#### Initial GNN + RL System
- General structure to understand how hydro network plays in ML
- How does features in GNN be used with DRL vice versa.
- This version doesn't have <b>phyiscs</b>
- Version 3, is to adjust most of the sort comings in this Architecture
    - Contains Simple Physics such as Friction Calculation, Pressure, Junctions, and Energy loss factor

In [7]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import networkx as nx
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

Device: cuda


#### Custom Synthetic Data
- <b>water_level</b>: The current water level at each node at each timestep. It is updated based on inflows (both natural and from neighboring nodes) and outflows (controlled by the valve).
- <b>inflow_rate</b>: The natural inflow rate into the node, modulated by a seasonal/cyclic pattern.
- <b>outflow_rate</b>: The outflow rate is a function of the valve position and the node's current water level.
- <b>valve_position</b>: The valve position for each node, controlling how much water flows out. It can be controlled by your policy network.


In [2]:
num_nodes = 100
num_edges = 35
time_steps = 100

seasonal_cycle_length = 1440  # Cyclic pattern
np.random.seed(42)

G = nx.gnm_random_graph(num_nodes, num_edges)
edge_list = np.array(G.edges())
num_edges = edge_list.shape[0]

# Flow Cap and Distances between nodes
flow_capacities = np.random.uniform(low=1.0, high=3.0, size=(num_edges,))
distances = np.random.uniform(low=0.5, high=5.0, size=(num_edges,))

# Simulating rainfall and evaporation with Synthetic Rainfall Cycle
time = np.arange(time_steps)
seasonal_pattern = np.sin(2 * np.pi * time / seasonal_cycle_length) * 0.2 + 0.3

initial_water_levels = np.random.uniform(low=0.5, high=2.0, size=num_nodes)

# Node interaction based on graph structure
water_levels = np.zeros((time_steps, num_nodes))
inflow_rates = np.zeros((time_steps, num_nodes))
outflow_rates = np.zeros((time_steps, num_nodes))
valve_positions = np.zeros((time_steps, num_nodes))

# Initial time step
water_levels[0, :] = initial_water_levels
valve_positions[0, :] = np.random.uniform(low=0.0, high=1.0, size=num_nodes)

# Inflow, outflow, and water level dynamics
for t in range(1, time_steps):
    inflow_rates[t, :] = seasonal_pattern[t % seasonal_cycle_length] + np.random.uniform(0.1, 0.5, size=num_nodes)
    valve_positions[t, :] = np.random.uniform(low=0.0, high=1.0, size=num_nodes)
    
    # Outflow rate depends on the valve position and previous node's water level
    outflow_rates[t, :] = valve_positions[t, :] * water_levels[t - 1, :]
    
    # Update water levels based on inflow and outflow
    for node in range(num_nodes):
        inflow_from_neighbors = 0
        for edge_idx, (src, tgt) in enumerate(edge_list):
            if tgt == node:
                inflow_from_neighbors += flow_capacities[edge_idx] * outflow_rates[t - 1, src] / distances[edge_idx]
        
        water_levels[t, node] = water_levels[t - 1, node] + inflow_rates[t, node] + inflow_from_neighbors - outflow_rates[t, node]
        water_levels[t, node] = np.clip(water_levels[t, node], 0, 2.0)

data = {
    'time_step': np.repeat(np.arange(time_steps), num_nodes),
    'node': np.tile(np.arange(num_nodes), time_steps),
    'water_level': water_levels.flatten(),
    'inflow_rate': inflow_rates.flatten(),
    'outflow_rate': outflow_rates.flatten(),
    'valve_position': valve_positions.flatten(),
}
df_nodes = pd.DataFrame(data)

df_edges = pd.DataFrame({
    'edge_index': np.arange(num_edges),
    'source_node': edge_list[:, 0],
    'target_node': edge_list[:, 1],
    'flow_capacity': flow_capacities,
    'distance': distances,
})

display(df_nodes.head())
display(df_edges.head())

flow_capacities = torch.tensor(flow_capacities, dtype=torch.float)
distances = torch.tensor(distances, dtype=torch.float)

,time_step,node,water_level,inflow_rate,outflow_rate,valve_position
0,0,0,1.658367,0.0,0.0,0.677564
1,0,1,0.798074,0.0,0.0,0.016588
2,0,2,0.508283,0.0,0.0,0.512093
3,0,3,1.723192,0.0,0.0,0.226496
4,0,4,1.560286,0.0,0.0,0.645173


,edge_index,source_node,target_node,flow_capacity,distance
0,0,4,23,1.749080,4.137788
1,1,5,52,2.901429,1.870762
2,2,5,79,2.463988,0.939525
3,3,5,46,2.197317,3.579049
4,4,6,13,1.312037,2.480686


##### Simple GNN Architecture
$$
h_v^{(k+1)} = \sigma \left( \sum_{u \in \mathcal{N}(v)} \frac{1}{\sqrt{d_u d_v}} W^{(k)} h_u^{(k)} \right)
$$

where $ h_v^{(k)} $ represents the node features at layer $ k $, and $ W^{(k)} $ is the weight matrix.

In [3]:
# NOTE GNN Simple Arch
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.3)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.3)
        x = self.conv3(x, edge_index)
        return x

##### Reinforcement Learning (Policy Network)
The policy network in the RL model maps the state to an action:

$$
\text{Action} = \text{PolicyNetwork}(State)
$$

Optimization Methods
- Switching Optimizer
- Hierarchical RL
- Aggergator Method (Could Try but from theory calculation it's not good)

In [23]:
# NOTE Simple DRL | Switch Optimizing (Local and Global)
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))

# Local control (Per-Node)
def local_control(node_representations, policy_network):
    actions = policy_network(node_representations)
    return actions

# Global control (Coordinate all nodes)
def global_control(node_representations, policy_network):
    graph_representation = torch.mean(node_representations, dim=0)
    global_action = policy_network(graph_representation)
    actions = global_action.expand(node_representations.shape[0], -1)
    return actions

# Switching mechanism for local and global control
def hybrid_control(node_representations, policy_network, global_threshold, current_water_levels):
    max_water_level = torch.max(current_water_levels)
    
    if max_water_level > global_threshold:
        print("Switching to Global Control!")
        actions = global_control(node_representations, policy_network)
    else:
        print("Using Local Control.")
        actions = local_control(node_representations, policy_network)    
    return actions

def environment_step(state, action, edge_index, flow_capacities):
    inflow_rate = state[:, 1]
    
    # print(f"Inflow Rate {inflow_rate.shape}")    
    # print(f"State shape: {state.shape}")
    # print(f"Initial action shape: {action.shape}")  
    
    if action.dim() == 2 and action.shape[0] < state.shape[0]:
        action = action.repeat(state.shape[0] // action.shape[0], 1)
    # print(f"Action shape after repeat: {action.shape}")  
    
    # outflow_rate = state[:, 2] + action.squeeze()
    outflow_rate = state[:, 2] + action[:, 0]
    # print(f"State Shape {state[:, 2].shape}", f"Action Shape {action.shape}")
    
    inflow_rate_new = inflow_rate.clone()
    for edge_idx, (src, tgt) in enumerate(edge_list):
        # print(f"Inflow Rate: {inflow_rate[tgt]}")
        # print(f"Outflow Rate: {outflow_rate[src]}")
        # print(f"Flow Cap {flow_capacities[edge_idx]}", f"Distance {distances[edge_idx]}")
        inflow_rate_new[tgt] += flow_capacities[edge_idx] * outflow_rate[src] / distances[edge_idx]
    
    outflow_rate = torch.clamp(outflow_rate, 0, 1)
    new_water_level = state[:, 0] + inflow_rate_new - outflow_rate
    new_water_level = torch.clamp(new_water_level, 0, 2)
    
    new_state = torch.stack((new_water_level, state[:, 1], outflow_rate, state[:, 3]), dim=1)
    overflow_pen = overflow_penalty(new_state)
    stability = stability_reward(new_state)
    reward = overflow_pen + stability
    
    return new_state, reward

def overflow_penalty(state):
    penalty = torch.clamp(state[:, 0] - 1.5, min=0)
    return -torch.sum(penalty) * 10  # Higher penalty for overflow

def stability_reward(state):
    ideal_level = 1.0
    return -torch.mean((state[:, 0] - ideal_level) ** 2)  # Minimize deviation from the ideal level

#### Next Steps (Might be bottlenecked by computational resources)
- MARL
- Advanced Reward System
- Testing different learning strats (PPO vs Q-Learning vs A2C)

In [5]:
# NOTE Simple but effective way to add X features (Could improve in later phases)
gnn_in_chan = 4  # aka. Num Input Feat (e.g, water_level, inflow_rate, outflow_rate, valve_position)
node_features = np.zeros((time_steps, num_nodes, gnn_in_chan))
for t in range(time_steps):
    for n in range(num_nodes):
        node_features[t, n, 0] = df_nodes.loc[(df_nodes['time_step'] == t) & (df_nodes['node'] == n), 'water_level'].values[0]
        node_features[t, n, 1] = df_nodes.loc[(df_nodes['time_step'] == t) & (df_nodes['node'] == n), 'inflow_rate'].values[0]
        node_features[t, n, 2] = df_nodes.loc[(df_nodes['time_step'] == t) & (df_nodes['node'] == n), 'outflow_rate'].values[0]
        node_features[t, n, 3] = df_nodes.loc[(df_nodes['time_step'] == t) & (df_nodes['node'] == n), 'valve_position'].values[0]
node_features_tensor = torch.tensor(node_features, dtype=torch.float)
# NOTE Will need LSTM for multi-step to deal with sequencial data: node_features_tensor.to(device)
# single_immediate_features = node_features[0] # NOTE for immediate condition (Snapshot)
# single_immediate_features_tensor = torch.tensor(single_immediate_features, dtype=torch.float).to(device)
# print(f"Node Features: {node_features.shape}")
# print(f"Single Node Features: {single_immediate_features.shape}")

# water_data = from_networkx(G)
# water_data.x = single_immediate_features_tensor
# water_data.edge_index = water_data.edge_index.to(device)

In [17]:
water_data = from_networkx(G)

train_steps = int(0.7 * time_steps)
val_steps = int(0.15 * time_steps)
test_steps = time_steps - train_steps - val_steps

train_node_features = node_features_tensor[:train_steps, :, :].reshape(-1, gnn_in_chan).to(device)
val_node_features = node_features_tensor[train_steps:train_steps+val_steps, :, :].reshape(-1, gnn_in_chan).to(device)
test_node_features = node_features_tensor[train_steps+val_steps:, :, :].reshape(-1, gnn_in_chan).to(device)

train_edge_index = water_data.edge_index[:, :train_steps]
val_edge_index = water_data.edge_index[:, train_steps:train_steps+val_steps]
test_edge_index = water_data.edge_index[:, train_steps+val_steps:]  

train_data = Data(x=train_node_features, edge_index=train_edge_index.to(device))
val_data = Data(x=val_node_features, edge_index=val_edge_index.to(device))
test_data = Data(x=test_node_features, edge_index=test_edge_index.to(device))


In [26]:
# NOTE New Training Method
gnn_out_chann = 4 # Num of features out
rl_out_channels = 4 # Controlling valve positions (one output per node)
hidden_channels = 128
gnn = GCN(in_channels=gnn_in_chan, hidden_channels=hidden_channels, out_channels=gnn_out_chann).to(device)
policy_network = PolicyNetwork(input_dim=gnn_out_chann, output_dim=rl_out_channels).to(device)
optimizer = torch.optim.Adam(list(gnn.parameters()) + list(policy_network.parameters()), lr=1e-3)
# schedule = ReduceLROnPlateau

num_episodes = 1_000
best_loss = float("inf")
train_losses = []
val_losses = []
es_threshold = 15
early_stoppping = 0

torch.autograd.set_detect_anomaly(True) # Inplace Error??
for episode in range(num_episodes):
    gnn.train()
    policy_network.train()
    
    # Training Phase
    train_current_water_levels = train_data.x[:, 0]
    train_global_threshold = torch.quantile(train_current_water_levels, 0.95)
    
    node_representations = gnn(train_data)
    actions = hybrid_control(node_representations, policy_network, train_global_threshold, train_current_water_levels)
    new_state, reward = environment_step(node_representations, actions, train_data.edge_index, flow_capacities)
    
    train_loss = -reward.mean()
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    train_losses.append(train_loss.item())
    
    # Validation Phase
    gnn.eval()
    policy_network.eval()
    with torch.no_grad():
        val_node_representations = gnn(val_data)
        val_actions = hybrid_control(val_node_representations, policy_network, train_global_threshold, val_data.x[:, 0])
        val_new_state, val_reward = environment_step(val_node_representations, val_actions, val_data.edge_index, flow_capacities)        
        val_loss = -val_reward.mean()
        val_losses.append(val_loss.item())
        
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping = 0 # Reset
        torch.save(gnn.state_dict(), 'best_gnn_model.pth')
        torch.save(policy_network.state_dict(), 'best_policy_network.pth')    
    elif val_loss > best_loss:
        early_stoppping += 1
        if early_stoppping == es_threshold:
            print(f'Early Stopping Triggered: Epoch {episode} | Train Loss: {train_loss.item()} | Val Loss: {val_loss.item()}')
            break
    print(f'Epoch {episode}: Train Loss: {train_loss.item()}, Val Loss: {val_loss.item()}')

gnn.load_state_dict(torch.load('best_gnn_model.pth'))
policy_network.load_state_dict(torch.load('best_policy_network.pth'))


Using Local Control.
Using Local Control.
Epoch 0: Train Loss: 0.9912256002426147, Val Loss: 0.9409996867179871
Using Local Control.
Using Local Control.
Epoch 1: Train Loss: 0.9449146389961243, Val Loss: 0.8389946222305298
Using Local Control.
Using Local Control.
Epoch 2: Train Loss: 0.8423213958740234, Val Loss: 0.65670245885849
Using Local Control.
Using Local Control.
Epoch 3: Train Loss: 0.6787669658660889, Val Loss: 0.48555704951286316
Using Local Control.
Using Local Control.
Epoch 4: Train Loss: 0.49235671758651733, Val Loss: 0.31304121017456055
Using Local Control.
Using Local Control.
Epoch 5: Train Loss: 1.396718144416809, Val Loss: 0.31006038188934326
Using Local Control.
Using Local Control.
Epoch 6: Train Loss: 0.3130315840244293, Val Loss: 0.2983452379703522
Using Local Control.
Using Local Control.
Epoch 7: Train Loss: 0.29199206829071045, Val Loss: 0.2716655731201172
Using Local Control.
Using Local Control.
Epoch 8: Train Loss: 0.26894474029541016, Val Loss: 0.231466

C:\Users\Jeff\AppData\Local\Temp\ipykernel_30828\1290655766.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  gnn.load_state_dict(torch.load('best_gnn_model.pth'))
C:\Use

<All keys matched successfully>

#### GAT
In the GAT layer, the node representations are updated using attention mechanisms:

$$
h_v' = \sum_{u \in \mathcal{N}(v)} \alpha_{vu} W h_u
$$

where $ \alpha_{vu} $ is the attention coefficient between node $ v $ and node $ u $.